In [1]:
import json
import os
import random

import open3d as o3d

import torch
import numpy as np

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
import sys 
sys.path.append('..')
import utils.misc as ws
import utils.data_utils
import utils.train_utils
import utils.eval_utils
import utils.mesh
import utils.dataset as d
import models.networks as arch


In [3]:
# DATA_SOURCE = '/home/ninad/Desktop/Docs/phd-res/proj-irvl-grasp-transfer/code/docker-data/output_dataset/'
# DATA_SOURCE = '/home/ninad/Desktop/multi-finger-grasping/output_dataset/'
DATA_SOURCE = '/mnt/Data/eval_dataset-8ycb-5gripper-10grasps/'

object_model_name = "003_cracker_box_google_16k_textured_scale_1000"

EXPERIMENTS_DIR = '../experiments/all5_003_dsdf_50_varcmap'
CHECKPOINT = 'latest'

split_filename = os.path.join(EXPERIMENTS_DIR, 'split_validation.json')
specs_filename = os.path.join(EXPERIMENTS_DIR, "specs.json")

LATENT_CODE_DIR = ws.latent_codes_subdir
reconstructions_subdir = ws.reconstructions_subdir
reconstruction_codes_subdir = ws.reconstruction_codes_subdir

# Load the object point cloud
model_pc_file = os.path.join(DATA_SOURCE, object_model_name, 'object_point_cloud.ply')
obj_pcd = o3d.io.read_point_cloud(model_pc_file)
gt_obj_pc_points = np.asarray(obj_pcd.points)
print(gt_obj_pc_points.shape)

# Load the normalization params:
norm_file = os.path.join(DATA_SOURCE, object_model_name, 'norm_params.npz')
norm_npz = np.load(norm_file)
offset = norm_npz['offset']
scale = norm_npz['scale']

# Normalize the object point cloud
gt_obj_pc_points = (gt_obj_pc_points - offset) / scale

(8194, 3)


In [4]:
specs = json.load(open(specs_filename))
latent_size = specs["CodeLength"]
gripper_weight = specs["GripperWeight"]
num_grippers = specs["NumGrippers"]

In [5]:
decoder = arch.dsdfDecoder(
    latent_size, 
    **specs["NetworkSpecs"]
    ).cuda()

decoder = torch.nn.DataParallel(decoder)

saved_model_state = torch.load(
    os.path.join(
        EXPERIMENTS_DIR, ws.model_params_subdir, CHECKPOINT + ".pth")
)

saved_model_epoch = saved_model_state["epoch"]

decoder.load_state_dict(saved_model_state["model_state_dict"])

decoder = decoder.module.cuda()


In [6]:
with open(split_filename, "r") as f:
    split = json.load(f)

# npz_filenames = utils.data_utils.dsdf_get_instance_filenames(
#     args.data_source, split)
cmap_f, grp_names, gpc_f, npz_filenames = utils.data_utils.get_instance_filelist(DATA_SOURCE, split)

# random.shuffle(npz_filenames) # WHY??? DISABLE THIS FOR CHECKING REPRODUCIBILITY

In [7]:
def get_reconstructed_code_filename(experiment_dir, epoch, ycb_model, gripper_name, instance_name):
    return os.path.join(
        experiment_dir,
        reconstructions_subdir,
        str(epoch),
        reconstruction_codes_subdir,
        f"{ycb_model}-sdf-{gripper_name}-{instance_name}.pth")
    # ycb_model + '-sdf-' + gripper_name + '-' + instance_name + ".pth")


In [16]:
index_to_select = random.randint(0, len(npz_filenames)-1)

npz = npz_filenames[index_to_select]
gripper = grp_names[index_to_select]
print(index_to_select, npz[-35:])
npz_instance_name = os.path.split(npz)[-1].split('.')[0]
lvec_file = get_reconstructed_code_filename(EXPERIMENTS_DIR, saved_model_epoch,
                                            object_model_name, gripper, npz_instance_name)
latent_vec = torch.load(lvec_file)
latent_vec = latent_vec.squeeze()
# latent_vec = latent_vecs[index_to_select].squeeze()
# print(latent_vec.shape)


# Load the point cloud corresponding to this grasp/gripper
grp_pc_file = utils.eval_utils.extract_pcfile_from_npzfile(npz)
grp_pcd = o3d.io.read_point_cloud(grp_pc_file)
gt_grp_pc_points = np.asarray(grp_pcd.points)
# Normalize the loaded points
gt_grp_pc_points = (gt_grp_pc_points - offset) / scale

print(gt_grp_pc_points.shape)



22 /sdf/panda_grasp/sdf_graspnum_2.npz
(224048, 3)


In [17]:
# with torch.no_grad():
#     queries, sdf_obj, sdf_grp = utils.eval_utils.eval_query_pc(decoder, latent_vec.cuda(), queries)

# Try with random queries instead:
with torch.no_grad():
    queries, sdf_obj, sdf_grp = utils.eval_utils.eval_random_query_pc(decoder, latent_vec.cuda(), num_samples=1000000)


In [18]:
EPS = 1e-4
gen_grp_points = queries[sdf_grp < EPS]
print(gen_grp_points.shape)

gen_obj_points = queries[sdf_obj < EPS]
print(gen_obj_points.shape)


torch.Size([11519, 3])
torch.Size([8549, 3])


In [19]:
utils.eval_utils.compute_pc_chamfer(gt_obj_pc_points, gen_obj_points)

(0.0011767164521415139, 0.00010762797866885798)

In [20]:
print(utils.eval_utils.compute_pc_chamfer(gt_grp_pc_points, gen_grp_points))

(0.009285194853763284, 0.00020036756071781643)


In [21]:
import pyrender
# Just pass the points you want to visualize
def plt_points_3d(pts):
    colors = np.zeros(pts.shape)
    cloud = pyrender.Mesh.from_points(pts, colors=colors)
    scene = pyrender.Scene()
    scene.add(cloud)
    viewer = pyrender.Viewer(scene, use_raymond_lighting=True, point_size=2)

# Pass the point as well as the sdf to see inside/outside points
def plt_points_sdf(pts, sdf, eps=1e-4):
    colors = np.zeros(pts.shape)
    colors[sdf < eps, 1] = 1
    colors[sdf > eps, 0] = 1
    cloud = pyrender.Mesh.from_points(pts, colors=colors)
    scene = pyrender.Scene()
    scene.add(cloud)
    viewer = pyrender.Viewer(scene, use_raymond_lighting=True, point_size=2)

def plt_points_sdf_compare(pts, sdf_gt, sdf_pred):
    colors = np.zeros(pts.shape)
    colors[sdf_gt < 0, 1] = 1
    colors[sdf_gt > 0, 0] = 1
    cloud = pyrender.Mesh.from_points(pts, colors=colors)
    scene = pyrender.Scene()
    scene.add(cloud)
    viewer = pyrender.Viewer(scene, use_raymond_lighting=True, point_size=2)
    
    colors2 = np.zeros(pts.shape)
    colors2[sdf_pred < 0, 1] = 1
    colors2[sdf_pred > 0, 0] = 1
    cloud2 = pyrender.Mesh.from_points(pts, colors=colors2)
    scene2 = pyrender.Scene()
    scene2.add(cloud2)
    viewer = pyrender.Viewer(scene2, use_raymond_lighting=True, point_size=2)

In [22]:
plt_points_3d(queries[sdf_grp < 1e-4])

In [23]:
plt_points_3d(queries[sdf_obj < 1e-4])